In [1]:
print(u'\U0001F430')

🐰


In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import itertools 
from collections import defaultdict 
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools

init_notebook_mode(connected=True)
%matplotlib inline 

#### Solving rabbit reproduction programming challenge

https://www.reddit.com/r/dailyprogrammer/comments/7s888w/20180122_challenge_348_easy_the_rabbit_problem/

In [3]:
def rabbits(init_male, init_female, target):
    
    rabbits = np.zeros((96,2))
    rabbits[2] = [init_male, init_female]
    birthrate = np.array([5, 9])
    month = 0
    
    while rabbits.sum() < target:
        fertile = rabbits[4:, 1].sum()
        newborns = fertile * birthrate
        rabbits = np.vstack([newborns, rabbits[:-1]])
        month += 1
        
    return month 

#### Simulating results and plotting relationship between input parameters and output 


In [52]:
%%time
test_target = range(1, 5000, 50)
test_female = range(1, 100, 2)
test_male = range(1, 100, 2)

results = defaultdict(list)


for i, (male, female, target) in enumerate(itertools.product(test_male, test_female, test_target)):
    if i % 100000 == 0: print(i, 'records complete') 
    results['male'].append(male)
    results['female'].append(female)
    results['target'].append(target)
    results['y'].append(rabbits(male, female, target))

0 records complete
100000 records complete
200000 records complete
CPU times: user 1min 35s, sys: 13.4 ms, total: 1min 35s
Wall time: 1min 37s


In [53]:
df = pd.DataFrame(results)

In [76]:
def plot_surface(df):
    data = [
        go.Surface(
            x=df.columns, 
            y=df.index, 
            z=df.as_matrix()
        )
    ]
    layout = go.Layout(
        title='Rabbits',
        scene = dict(          
            xaxis = dict(title=df.columns.name),
            yaxis = dict(title=df.index.name),
            zaxis = dict(title='Y'),
        ),
        autosize=False,
        width=750,
        height=500,
        margin=dict(
            l=10,
            r=10,
            b=50,
            t=50
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

We can only plot 2 dimensions vs. target output so creating three plots where we fix one of our input variables and plotting the other two vs. output "Y" (number of days needed to reach target)

In [77]:
df_female_target = df.loc[df.male == 1, ['female','target', 'y']].pivot(index='female', columns='target', values='y')
df_male_target = df.loc[df.female == 1, ['male','target', 'y']].pivot(index='male', columns='target', values='y') 
df_male_female = df.loc[df.target == 1001, ['female','male', 'y']].pivot(index='female', columns='male', values='y') 

In [78]:
plot_surface(df_female_target) 

In [79]:
plot_surface(df_male_target) 

In [80]:
plot_surface(df_male_female) 

These plots show us quite clearly that both the target and inital number of females have a strong influcence on the outcome (number of months required to reach target). However the inital number of males has very little impact on the result. This is because each female can reproduce and increase the total rabbit count by 14 every day, whereas males cannot increase the population and so increasing the initial number of males potentially has a negligible impact, depending on what your target is.